In [78]:
%env UV_LINK_MODE=copy
!uv add datamodel-code-generator datamodel-code-generator[graphql,http] tomli
!uv sync

env: UV_LINK_MODE=copy


Resolved 121 packages in 0.58ms
error: failed to remove file `D:\PythonProject\.venv\Lib\site-packages\zmq/backend/cython/_zmq.cp313-win_amd64.pyd`: Access is denied. (os error 5)
Resolved 121 packages in 0.64ms
error: failed to remove file `D:\PythonProject\.venv\Lib\site-packages\zmq/backend/cython/_zmq.cp313-win_amd64.pyd`: Access is denied. (os error 5)


In [79]:
from pathlib import Path
from tempfile import TemporaryDirectory
from datamodel_code_generator import InputFileType, generate, PythonVersion
from datamodel_code_generator import DataModelType
from datamodel_code_generator.http import get_body
from datamodel_code_generator.model.pydantic_v2 import UnionMode
from pprint import pprint, pformat
from json import loads
import tomli

In [80]:
pyproject = tomli.loads(Path('pyproject.toml').read_text())
datamodel_codegen_config = pyproject.get('tool', {}).get('datamodel-codegen', {})
pprint(datamodel_codegen_config)

{'allow_population_by_field_name': True,
 'capitalise_enum_members': True,
 'collapse_root_models': True,
 'field_constraints': True,
 'field_include_all_keys': True,
 'formatters': ['ruff-check', 'ruff-format'],
 'reuse_model': True,
 'snake_case_field': True,
 'strip_default_none': False,
 'use_annotated': True,
 'use_double_quotes': True,
 'use_field_description': True,
 'use_schema_description': True,
 'use_subclass_enum': True,
 'use_title_as_name': True,
 'use_union_operator': True,
 'use_unique_items_as_set': True}


In [81]:
CARD_JSON_SCHEMA_URL = r"https://raw.githubusercontent.com/Kamalisk/arkhamdb-json-data/refs/heads/master/schema/card_schema.json"
CARD_JSON_SCHEMA = get_body(CARD_JSON_SCHEMA_URL)
print(CARD_JSON_SCHEMA)

{
    "additionalProperties": true,
    "properties": {
        "back_text": {
            "minLength": 1,
            "type": "string"
        },
        "back_flavor": {
            "minLength": 0,
            "type": "string"
        },
        "code": {
            "pattern": "^[0-9a-z]{5}[0-9a-z]?$",
            "type": "string"
        },
        "cost": {
            "oneOf": [
                {
                    "minimum": -2,
                    "type": "integer"
                },
                {
                    "type": "null"
                }
            ]
        },
        "deck_requirements": {
            "oneOf": [
                {
                    "minLength": 0,
                    "type": "string"
                },
                {
                    "type": "null"
                }
            ]
        },
        "deck_options": {
            "oneOf": [
                {
                    "type": "array"
                },
                {
      

In [82]:

# Save the Pydantic model to the path 'models/<schema_type>.py' where <schema_type> is whatever is between the last '/' and the word 'schema' in the URL
schema_model_name = CARD_JSON_SCHEMA_URL.split('/')[-1].rstrip('_schema.json')
model_output_path = Path('models') / Path(schema_model_name)
model_output_path= model_output_path.with_suffix('.py').absolute()
generate(
    CARD_JSON_SCHEMA,
    output=model_output_path,
    output_model_type=DataModelType.PydanticV2BaseModel,
    input_file_type=InputFileType.JsonSchema,
    target_python_version=PythonVersion.PY_313,
    union_mode=UnionMode.smart,
    **datamodel_codegen_config
)
card_model: str = model_output_path.read_text()
print(card_model.replace(r'\n', ''))


# generated by datamodel-codegen:
#   filename:  <stdin>
#   timestamp: 2025-04-04T16:36:33+00:00

from __future__ import annotations
from typing import Annotated, Any, List
from pydantic import BaseModel, ConfigDict, Field, RootModel


class Cost(RootModel[int]):
    model_config = ConfigDict(
        populate_by_name=True,
    )
    root: Annotated[int, Field(ge=-2)]


class DeckRequirements(RootModel[str]):
    model_config = ConfigDict(
        populate_by_name=True,
    )
    root: Annotated[str, Field(min_length=0)]


class EnemyDamage(Cost):
    pass


class EnemyEvade(Cost):
    pass


class EnemyFight(Cost):
    pass


class EnemyHorror(Cost):
    pass


class Health(Cost):
    pass


class Sanity(Cost):
    pass


class SubtypeCode(DeckRequirements):
    pass


class Victory(RootModel[int]):
    model_config = ConfigDict(
        populate_by_name=True,
    )
    root: Annotated[int, Field(ge=0)]


class Xp(Victory):
    pass


class Model(BaseModel):
    model_config = Config